In [1]:
import pandas as pd
import numpy as np
import altair as alt

## Data ingestion and formating

In [2]:
df = pd.read_csv('../data/emotion_forced_choice_uw_students.csv')
df_labels = pd.read_csv('../data/emotion_labels.csv')

In [3]:
df = df.iloc[5:, :] # filter out test rows

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 5 to 85
Columns: 220 entries, StartDate to Q195.1
dtypes: object(220)
memory usage: 139.3+ KB


In [5]:
df = df[df['Finished'] =='True'] # filter out incomplete surveys

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 5 to 85
Columns: 220 entries, StartDate to Q195.1
dtypes: object(220)
memory usage: 88.1+ KB


In [7]:
df.rename(columns={
    'Q1.2': 'sex',
    'Q1.3_1': 'age',
    'Q1.4': 'ethnicity',
    'Q1.5': 'formal education',
    'Q1.6_1': 'income'}, inplace=True)

In [8]:
df.to_csv('../clean_data/forced_choice_emotion_uw_students.csv', index=False)

### Words by frequency for all images (ranking)

In [9]:
def count_freq_labels(df, X="all" ):
    if X == "all":
        df_counts = df.stack().reset_index(drop=True).value_counts() # stack as series
        df_counts = df_counts.to_frame('counts') # get value_counts as df
        df_counts['emotion'] = df_counts.index # get index as col
    else:
        df_counts = df[X].reset_index(drop=True).value_counts() # stack as series
        df_counts = df_counts.to_frame('counts') # get value_counts as df
        df_counts[X] = df_counts.index # get index as col

    df_counts = df_counts.reset_index(drop=True) # clean index
    df_counts['percent'] = df_counts['counts'] / df_counts['counts'].sum() # compute percentage
    return df_counts

In [10]:
def simple_per_bar(
    df, title='Title', X='percent:Q', Y='emotion:N', \
    width=450, height=250, sort='-x', \
    text_size = 12, label_size = 11, title_size=12, \
    emotion='Some', color1='#0570b0', color2='orange'):
    
    bars = alt.Chart(df, title=title).mark_bar().encode(
        alt.X(X, axis=alt.Axis(format='.0%')),
        y=alt.Y(Y, sort=sort), 
        color=alt.condition(
            alt.datum.emotion == emotion,
            alt.value(color2),
            alt.value(color1)
        ))
    
    text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
    fontSize=text_size,
    fontWeight="bold"
    ).encode(
        alt.Text(X, format='.1%')
    )
    
    chart = (bars + text).configure_axis(
            labelFontSize=label_size,
            titleFontSize=title_size).properties(
                width=width, 
                height=height).display(renderer='svg')
    
    
    return chart

In [11]:
def simple_count_bar(
    df, title='Title', X='counts:Q', Y='emotion:N', \
    width=450, height=250, sort='-x', \
    text_size = 12, label_size = 11, title_size=12,
    emotion='Some', color1='#0570b0', color2='#orange'):
    
    bars = alt.Chart(df, title=title).mark_bar().encode(
        alt.X(X),
        y=alt.Y(Y, sort=sort), 
        color=alt.condition(
            alt.datum.emotion == emotion,
            alt.value(color2),
            alt.value(color1)
        ))
    
    text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
    fontSize=text_size,
    fontWeight="bold"
    ).encode(
        alt.Text(X)
    )
    
    chart = (bars + text).configure_axis(
            labelFontSize=label_size,
            titleFontSize=title_size).properties(
                width=width, 
                height=height).display(renderer='svg')
    
    
    return chart

## Demographics

In [12]:
source = count_freq_labels(df, X="sex") 
title = 'Sex | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'sex:N'
w, h= 450, 100
txs, ls, ts = 12, 12, 12

chart_sex = simple_per_bar(source, title=title, X=X, Y=Y, width=w, height=h, text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

In [13]:
source = count_freq_labels(df, X="age") 
title = 'Age | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'age:N'
w, h= 450, 150
txs, ls, ts = 12, 11, 12

chart_age = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h, \
    text_size = txs, label_size = ls, title_size=ts)
chart_age

alt.LayerChart(...)

In [14]:
source = count_freq_labels(df, X="ethnicity") 
title = 'Ethnicity | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'ethnicity:N'
w, h= 450, 150
txs, ls, ts = 12, 11, 12

chart_ethnicity= simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h, \
    text_size = txs, label_size = ls, title_size=ts)
chart_ethnicity

alt.LayerChart(...)

In [15]:
source = count_freq_labels(df, X="formal education") 
title = 'Formal education | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'formal education:N'
w, h= 450, 150
txs, ls, ts = 12, 11, 12

chart_formal_education= simple_per_bar(
    source, title=title, X=X, Y=Y, width=w, height=h, text_size = txs, label_size = ls, title_size=ts)
chart_formal_education

alt.LayerChart(...)

## Overall results

In [16]:
df_emo_answers = df.loc[:, 'Q2.1':'Q195.1'] # subset photos

In [17]:
source = count_freq_labels(df_emo_answers, X="all") 
title = 'Labels frequency | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12

chart_overall_per = simple_per_bar(
    source, title=title, X=X, Y=Y, width=w, height=h, text_size = txs, label_size = ls, title_size=ts)
chart_overall_per

alt.LayerChart(...)

In [18]:
source = count_freq_labels(df_emo_answers, X="all") 
title = 'Labels frequency | n = '+ source['counts'].sum().astype(str)
X, Y = 'counts:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12

chart_overall_count = simple_count_bar(
    source, title=title, X=X, Y=Y, width=w, height=h, text_size = txs, label_size = ls, title_size=ts)
chart_overall_count

alt.LayerChart(...)

## Most frequently used word for each emotion category (grouping pictures by expected-emotion)

In [19]:
def emotion_df_formated(df_emo_answers, emotion_label):
    df_emo_cat = df_emo_answers.copy() 
    df_emo_cat_t = df_emo_cat.T # transpose
    df_emo_cat_t['photo_id'] = df_emo_cat_t.index # get index as col
    df_emo_cat_t = df_emo_cat_t.reset_index(drop=True) # clean index
    df_emo_cat_t_labels = pd.concat([df_emo_cat_t, df_labels], axis=1) # add metadata cols
    df_label =  df_emo_cat_t_labels[df_emo_cat_t_labels['label'] == emotion_label]
    return df_label

In [20]:
emotion = 'anger'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_anger = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h, emotion=emotion.capitalize(), \
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Disgust

In [21]:
emotion = 'disgust'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_disgust = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Fear

In [22]:
emotion = 'fear'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_fear = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Surprise

In [23]:
emotion = 'surprise'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_surprise = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(), \
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Happiness

In [24]:
emotion = 'happiness'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_happiness = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Sadness

In [25]:
emotion = 'sadness'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_sadness = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Uncertain

In [26]:
emotion = 'uncertain'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_uncertain = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Neutral

In [27]:
emotion = 'neutral'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_neutral = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

## Most frequently used word for each emotion category (grouping pictures by expected-emotion), by ethnicity group

In [28]:
def emotion_df_formated_et(df_emo_answers, emotion_label, ethnicity):
    df_emo_cat = df_emo_answers.copy() 
    df_emo_cat_t = df_emo_cat.T # transpose
    df_emo_cat_t['photo_id'] = df_emo_cat_t.index # get index as col
    df_emo_cat_t = df_emo_cat_t.reset_index(drop=True) # clean index
    df_emo_cat_t_labels = pd.concat([df_emo_cat_t, df_labels], axis=1) # add metadata cols
    df_label =  df_emo_cat_t_labels[(df_emo_cat_t_labels['label'] == emotion_label) & (df_emo_cat_t_labels['ethnicity'] == ethnicity)]
    return df_label

In [29]:
def wrapper_chart_emotion(df_emo_answers, emotion, ethnicity):
    df = emotion_df_formated_et(df_emo_answers, emotion,  ethnicity) # subset 'anger' rows
    df_formated_ans = df.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
    df_count = count_freq_labels(df_formated_ans) # count label freq
    chart = simple_per_bar(
        df_count, \
        title='Expected label: '+ emotion + ' | n = '+ df_count['counts'].sum().astype(str), \
         emotion=emotion.capitalize())
    return chart

## Anger

In [41]:
chart_anger_bipoc = wrapper_chart_emotion(df_emo_answers, 'anger', 'bipoc')
chart_anger_white = wrapper_chart_emotion(df_emo_answers, 'anger', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Disgust

In [32]:
chart_disgust_bipoc = wrapper_chart_emotion(df_emo_answers, 'disgust', 'bipoc')
chart_disgust_white = wrapper_chart_emotion(df_emo_answers, 'disgust', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Fear

In [33]:
chart_fear_bipoc = wrapper_chart_emotion(df_emo_answers, 'fear', 'bipoc')
chart_fear_white = wrapper_chart_emotion(df_emo_answers, 'fear', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Surprise

In [34]:
chart_surprise_bipoc= wrapper_chart_emotion(df_emo_answers, 'surprise', 'bipoc')
chart_surprise_white = wrapper_chart_emotion(df_emo_answers, 'surprise', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Happiness

In [35]:
chart_happiness_bipoc = wrapper_chart_emotion(df_emo_answers, 'happiness', 'bipoc')
chart_happiness_white = wrapper_chart_emotion(df_emo_answers, 'happiness', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Sadness

In [36]:
chart_sadness_bipoc = wrapper_chart_emotion(df_emo_answers, 'sadness', 'bipoc')
chart_sadness_white = wrapper_chart_emotion(df_emo_answers, 'sadness', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Neutral

In [37]:
chart_neutral_bipoc = wrapper_chart_emotion(df_emo_answers, 'neutral', 'bipoc')
chart_neutral_white = wrapper_chart_emotion(df_emo_answers, 'neutral', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Uncertain/Other

In [38]:
chart_uncertain_bipoc = wrapper_chart_emotion(df_emo_answers, 'uncertain', 'bipoc')
chart_uncertain_white = wrapper_chart_emotion(df_emo_answers, 'uncertain', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

In [39]:
count_freq_labels(df_emo_answers, X="all")['emotion'].tolist() 

['Happiness',
 'Neutral',
 'Surprise',
 'Sadness',
 'Disgust',
 'Anger',
 'Other',
 'Fear',
 'Uncertain']